In [5]:
notebook_id = 101

<a id="ID_top"></a>
## Country data organisation

This workflow generates (part1), maintains (part2) and exports (part3)

Export: one .csv file, with all gravity dataset iso_3 codes, matched against all UN COMTRADE (except "world", and the BRI countries)

#### Notebook sections:
    
|| [0| Default imports](#ID_top) || [1|Part1 Reference table generation](#ID_part1) || [2|Part2 Table maintanence](#ID_part2) || [3|Part3 Table export](#ID_part3) || 

#### Import all packages that could be required

In [4]:
# %load s_package_import.py
# package library, use to ensure consistency across notebooks, refresh periodically
# general packages
import os # use with os.listdir(_path_)
import requests
import csv
import time
from datetime import datetime
from shutil import copyfile

#temp check
#from shutil import make_archive
#import zipfile

# data analysis packages
import pandas as pd
pd.options.display.max_columns = None # don't truncate columns
#pd.options.display.max_rows = None

import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import descartes

# custom scripts
import s_file_export
import s_filepaths
import s_un_comtrade_extract as s_un
import s_adj_matrix_plot

#=== network analysis
import networkx as nx
#=== gavity modelling
import gme as gme


#### Import module and declare path variables
`import s_filepaths.py`

In [6]:
# import ref file
import s_filepaths

# declare local variables to work with
path_raw = s_filepaths.path_raw
path_raw_dl = s_filepaths.path_raw_dl
path_store = s_filepaths.path_store
path_live = s_filepaths.path_live

<a id="ID_part1"></a>
### Part 1
|| [0| Default imports](#ID_top) || [1|Part1 Reference table generation](#ID_part1) || [2|Part2 Table maintanence](#ID_part2) || [3|Part3 Table export](#ID_part3) || 

**Load UN complete country reference list**

In [127]:
# load data frame
df_uncom_countries = pd.read_csv(f"{path_live}input_un_com_2015.csv.gzip",
                               compression="gzip",index_col="Unnamed: 0")

In [140]:
# See all columns and select those to keep
print(df_uncom_countries.columns)
column_keep = ["pt3ISO","ptCode"]

Index(['rtCode', 'rt3ISO', 'rtTitle', 'ptCode', 'pt3ISO', 'ptTitle', 'period',
       'rgDesc', 'yr', 'rgCode', 'cmdCode', 'TradeValue', 'periodDesc',
       'pfCode', 'cmdDescE'],
      dtype='object')


Select which columns to keep: 
- ptCode - to get UN code for country
- pt3ISO - 3 letter ISO for country to match with other datasets
- ptTitle - full UN country name

In [142]:
# trim down data frame to only key columns and rename, 
# finally drop any territories / areas without a iso_3 code.
df_un_clean = df_uncom_countries.loc[:,column_keep].drop_duplicates().reset_index(drop = True).dropna().copy()
column_names = ["iso_3","un_code"]

df_un_clean.columns = column_names
print(f"UN COMTRADE country count | {len(df_un_clean.iso_3.unique())}")

df_un_clean["un_flag"] = 1

df_un_clean.head()

UN COMTRADE country count | 234


iso_3  un_code  un_flag
0   WLD        0        1
1   AFG        4        1
2   DZA       12        1
3   ATG       28        1
4   ARG       32        1

**Load in belt and road initiative countries**

In [143]:
# View all files in live directory
print(os.listdir(f"{path_live}"))

['input_test.csv.gzip', 'input_un_com_2013.csv.gzip', 'input_un_com_2012.csv.gzip', 'input_un_com_2006-2009.csv.gzip', '.DS_Store', 'input_un_codes_ref.csv.gzip', 'input_bri_countries_manual_2020.csv.gzip', 'input_un_com_2016-2019.csv.gzip', 'input_un_com_2002-2005.csv.gzip', 'input_un_com_2014.csv.gzip', 'input_un_com_2015.csv.gzip', 'input_bri_countries_Dumor_Yao.csv.gzip', '2_raw_explainer_doc.md', 'input_dynamic_gravity.csv.gzip', 'input_un_com_2010_merged_ref.csv.gzip', 'input_gme_data_joined.csv.gzip', 'input_un_sample.csv.gzip']


In [144]:
# load data frame
df_bri_countries_raw = pd.read_csv(f"{path_live}input_bri_countries_manual_2020.csv.gzip",
                               compression="gzip",index_col="Unnamed: 0")
df_bri_countries_raw.head()

Country                      Region      Income category  \
0          Afghanistan                  South Asia           Low income   
1              Albania       Europe & Central Asia  Upper middle income   
2              Algeria  Middle East & North Africa  Upper middle income   
3               Angola          Sub-Saharan Africa  Lower middle income   
4  Antigua and Barbuda   Latin America & Caribbean          High income   

   Year MOU sign                                          MOU link  \
0         2016.0                                               NaN   
1         2017.0                                               NaN   
2         2018.0                                               NaN   
3         2018.0                                               NaN   
4         2018.0  https://eng.yidaiyilu.gov.cn/qwyw/rdxw/57191.htm   

                                           Comment iso_3  
0  https://eng.yidaiyilu.gov.cn/zchj/sbwj/1425.htm   AFG  
1                                              NaN   ALB  
2                                              NaN   DZA  
3                                              NaN   AGO  
4                                              NaN   ATG

In [98]:
df_bri_countries_raw[df_bri_countries_raw.iso_3 == "NGA"]

Country              Region      Income category  Year MOU sign MOU link  \
87  Nigeria  Sub-Saharan Africa  Lower middle income         2018.0      NaN   

   Comment iso_3  
87     NaN   NGA

In [116]:
# trim dataframe
df_bri_countries = df_bri_countries_raw.loc[:,["iso_3","Year MOU sign"]]
# rename columns 
df_bri_countries.columns = ["iso_3","bri_year"]
df_bri_countries["bri_data"] = 1
print(f"BRI country count | {len(df_bri_countries.iso_3.unique())}")
df_bri_countries.head()

BRI country count | 138


iso_3  bri_year  bri_data
0   AFG    2016.0         1
1   ALB    2017.0         1
2   DZA    2018.0         1
3   AGO    2018.0         1
4   ATG    2018.0         1

**Load Gravity dataset**

In [72]:
# load gravity dataset
file_name = "input_dynamic_gravity.csv.gzip"
df_grav_raw = pd.read_csv(f"{path_live}{file_name}",compression="gzip")
#df_grav_raw.head()

In [117]:
column_keep = ["iso3_d","country_d","lat_d","lng_d"]
df_grav = df_grav_raw.loc[:,column_keep].drop_duplicates().reset_index(drop = True).copy()

column_names = ["iso_3","country_name","lat","long"]
df_grav.columns = column_names
print(f"Gravity country count | {len(df_grav.iso_3.unique())}")
df_grav.tail()

Gravity country count | 253


iso_3  country_name        lat        long
248   WSM         Samoa -13.841545 -171.738650
249   YEM         Yemen  14.599034   45.970097
250   ZAF  South Africa -29.420467   25.905807
251   ZMB        Zambia -13.461213   27.676571
252   ZWE      Zimbabwe -18.929132   29.789742

<a id="ID_part2"></a>
### Part 2 Merge and any other transformation
|| [0| Default imports](#ID_top) || [1|Part1 Reference table generation](#ID_part1) || [2|Part2 Table maintanence](#ID_part2) || [3|Part3 Table export](#ID_part3) || 

Use gravity dataset with highest iso_3 country count.
Make sure all matched.

In [118]:
# Join dataframes
dataframes = [df_grav,df_un_clean,df_bri_countries]

In [148]:
# testing joins (all 234 un data points joined | all 138 BRI joined)
print("BRI",len(df_grav.merge(df_bri_countries, on = "iso_3",how = "inner")))
print("UN",len(df_grav.merge(df_un_clean, on = "iso_3",how = "inner"))) # missing one ["WLD"]

BRI 138
UN 233


In [146]:
df_master = df_grav.merge(df_bri_countries, on = "iso_3",how = "left")
df_master = df_master.merge(df_un_clean, on = "iso_3",how = "left")
df_master.tail()

iso_3  country_name        lat        long  bri_year  bri_data  un_code  \
248   WSM         Samoa -13.841545 -171.738650    2018.0       1.0    882.0   
249   YEM         Yemen  14.599034   45.970097    2017.0       1.0    887.0   
250   ZAF  South Africa -29.420467   25.905807    2015.0       1.0    710.0   
251   ZMB        Zambia -13.461213   27.676571    2018.0       1.0    894.0   
252   ZWE      Zimbabwe -18.929132   29.789742    2018.0       1.0    716.0   

     un_flag  
248      1.0  
249      1.0  
250      1.0  
251      1.0  
252      1.0

In [147]:
df_master.describe()

lat        long     bri_year  bri_data     un_code  un_flag
count  252.000000  252.000000   132.000000     138.0  233.000000    233.0
mean    16.790338   12.884589  2017.159091       1.0  428.334764      1.0
std     26.297986   73.370325     1.562097       0.0  252.774057      0.0
min    -69.247757 -176.183330  2013.000000       1.0    4.000000      1.0
25%      1.249773  -49.846912  2017.000000       1.0  212.000000      1.0
50%     16.039415   16.301794  2018.000000       1.0  428.000000      1.0
75%     38.927057   47.003645  2018.000000       1.0  646.000000      1.0
max     78.216682  179.216640  2020.000000       1.0  894.000000      1.0

In [150]:
#df_master[df_master.iso_3 == "NGA"]
#df_master.iso_3.value_counts().head() # just a check for uniques

<a id="ID_part3"></a>
### Part 3 | Export
|| [0| Default imports](#ID_top) || [1|Part1](#ID_part1) || [2|Part2](#ID_part2) || [3|Part3](#ID_part3) || [4|Part4](#ID_part4) || [5|Part5](#ID_part5) ||

In [152]:
file_name = f"{notebook_id}_master_country_table"
s_file_export.f_df_export(df_master,file_name)

Export | ../Data/1_raw_processed_backup/store_101_master_country_table_20200726_2144.csv | COMPLETE
COPY   | ../Data/2_raw_processed_input/input_101_master_country_table.csv.gzip | COMPLETE
